In [ ]:
# Kode Training YOLOv12n
import os
from ultralytics import YOLO
import yaml
import torch

def verify_dataset_structure(dataset_path):
    """
    Verifikasi struktur dataset dan format label
    """
    print("Memverifikasi struktur dataset...")
    
    issues = []
    stats = {'train': {}, 'val': {}, 'test': {}}
    
    for split in ['train', 'val', 'test']:
        split_path = os.path.join(dataset_path, split)
        img_path = os.path.join(split_path, 'images')
        lbl_path = os.path.join(split_path, 'labels')
        
        if os.path.exists(img_path):
            img_files = [f for f in os.listdir(img_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
            stats[split]['images'] = len(img_files)
            
            if os.path.exists(lbl_path):
                lbl_files = [f for f in os.listdir(lbl_path) if f.endswith('.txt')]
                stats[split]['labels'] = len(lbl_files)
                
                
                if lbl_files:
                    sample_label = os.path.join(lbl_path, lbl_files[0])
                    with open(sample_label, 'r') as f:
                        first_line = f.readline().strip()
                        if first_line:
                            parts = first_line.split()
                            if len(parts) != 5:
                                issues.append(f"Format label tidak valid di {split}/labels (harus: class_id x y w h)")
                            else:
                                try:
                                    class_id = int(parts[0])
                                    if class_id < 0 or class_id > 4:
                                        issues.append(f"Class ID di luar range 0-4 ditemukan: {class_id}")
                                except ValueError:
                                    issues.append(f"Class ID bukan integer di {split}/labels")
            else:
                if stats[split].get('images', 0) > 0:
                    issues.append(f"Folder {split}/labels tidak ditemukan")
    
 
    print("\nStatistik Dataset:")
    for split, data in stats.items():
        if 'images' in data:
            print(f"  {split:5}: {data.get('images', 0):4} images, {data.get('labels', 0):4} labels")
    

    if issues:
        print("\nMasalah yang ditemukan:")
        for issue in issues:
            print(f"  {issue}")
    
    return stats, issues

def create_yaml_config(dataset_path):
    """
    Buat file konfigurasi YAML untuk YOLO dengan multi-class detection
    """
   
    class_names = ['atas', 'depan', 'kanan', 'kiri', 'bawah']
    
    
    stats, issues = verify_dataset_structure(dataset_path)
    
    
    available_splits = []
    for split in ['train', 'val', 'test']:
        if stats.get(split, {}).get('images', 0) > 0:
            available_splits.append(split)
    
    if 'train' not in available_splits:
        raise FileNotFoundError("Folder train/images tidak ditemukan atau kosong!")
    if 'val' not in available_splits:
        print("Folder val tidak ditemukan, akan menggunakan train untuk validasi")
    
    
    config = {
        'path': os.path.abspath(dataset_path),
        'train': 'train/images',
        'val': 'val/images' if 'val' in available_splits else 'train/images',
        'nc': len(class_names),  
        'names': class_names  
    }
    
    
    if 'test' in available_splits:
        config['test'] = 'test/images'
    
    
    yaml_path = os.path.join(dataset_path, 'dataset.yaml')
    
    print(f"\nMembuat file YAML di: {yaml_path}")
    with open(yaml_path, 'w') as f:
        yaml.dump(config, f, default_flow_style=False, sort_keys=False)
    
   
    print("\nIsi file dataset.yaml:")
    print("=" * 50)
    with open(yaml_path, 'r') as f:
        print(f.read())
    print("=" * 50)
    
    return yaml_path

def check_gpu():
    """
    Cek ketersediaan GPU dan informasi VRAM
    """
    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
        print(f"GPU terdeteksi: {gpu_name}")
        print(f"VRAM: {gpu_memory:.1f} GB")
        
        # Rekomendasi batch size berdasarkan VRAM
        if gpu_memory < 4:
            recommended_batch = 8
        elif gpu_memory < 8:
            recommended_batch = 16
        elif gpu_memory < 12:
            recommended_batch = 24
        else:
            recommended_batch = 32
        
        print(f"Rekomendasi batch size: {recommended_batch}")
        return 'cuda', recommended_batch
    else:
        print("GPU tidak terdeteksi, menggunakan CPU (training akan lambat)")
        return 'cpu', 4

def train_yolo_model(yaml_path, model_name='yolov8n.pt', epochs=100, batch_size=None):
    """
    Training YOLO model untuk multi-class face detection
    """
    
    device, recommended_batch = check_gpu()
    if batch_size is None:
        batch_size = recommended_batch
    
    print(f"\nMemulai training dengan konfigurasi:")
    print(f"  Model: {model_name}")
    print(f"  Device: {device}")
    print(f"  Batch size: {batch_size}")
    print(f"  Epochs: {epochs}")
    
    
    model = YOLO(model_name)
    
    
    results = model.train(
        data=yaml_path,
        epochs=epochs,
        imgsz=320, 
        batch=batch_size,
        name='yolo_multipose_face',
        project='runs/detect',
        save=True,
        save_period=10, 
        cache=True,  
        device=device,
        workers=4, 
        patience=50,  
        optimizer='AdamW', 
        verbose=True,
        seed=42,
        deterministic=True,
        single_cls=False, 
        rect=False,
        cos_lr=False,
        close_mosaic=10,
        resume=False,  
        amp=True,  
        fraction=1.0,
        profile=False,
        
        hsv_h=0.015, 
        hsv_s=0.7,    
        hsv_v=0.4,  
        degrees=10.0,  
        translate=0.1,  
        scale=0.5,     
        shear=5.0,     
        perspective=0.0, 
        flipud=0.0,    
        fliplr=0.0,    
        mosaic=0.5,    
        mixup=0.0,    
        copy_paste=0.0,  
        
        lr0=0.01,     
        lrf=0.01,      
        momentum=0.937,
        weight_decay=0.0005,
        warmup_epochs=3.0,
        warmup_momentum=0.8,
        warmup_bias_lr=0.1,
        
        box=7.5,       
        cls=0.5,       
        dfl=1.5,       
                
        plots=True,
        val=True,
        iou=0.5,       
        max_det=100    
    )
    
    return results

def main():
    """
    Main function untuk menjalankan training
    """
    
    
    dataset_path = r"dataset_split" 
    
    
    model_name = 'yolo12n.pt'  
    
    
    epochs = 100  
    batch_size = None 
    
   
    
    print("=" * 60)
    print("YOLO Multi-Pose Face Detection Training")
    print("=" * 60)
    print(f"Classes: atas, bawah, depan, kanan, kiri")
    print(f"Dataset: {dataset_path}")
    print(f"Model: {model_name}")
    print("=" * 60)
    
    try:

        if not os.path.exists(dataset_path):
            print(f"Error: Folder dataset tidak ditemukan: {dataset_path}")
            print("Pastikan path sudah benar dan folder exists")
            return
        
 
        print("\nStep 1: Membuat konfigurasi YAML...")
        yaml_path = create_yaml_config(dataset_path)
        

        print("\nStep 2: Memulai training...")
        results = train_yolo_model(yaml_path, model_name, epochs, batch_size)
        
        print("\n" + "=" * 60)
        print("TRAINING SELESAI!")
        print("=" * 60)
        print(f"Model tersimpan di: runs/detect/yolo_multipose_face/weights/")
        print(f"  - best.pt: Model dengan performa terbaik")
        print(f"  - last.pt: Model dari epoch terakhir")
        print(f"Grafik dan metrics: runs/detect/yolo_multipose_face/")
        print("\n Untuk menggunakan model:")
        print("   from ultralytics import YOLO")
        print("   model = YOLO('runs/detect/yolo_multipose_face/weights/best.pt')")
        print("   results = model('path/to/image.jpg')")
        print("=" * 60)
        
    
    except Exception as e:
        print(f"\nError tidak terduga: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

In [ ]:
# Melakukan Quantitasi
from ultralytics import YOLO

model = YOLO("runs/detect/yolo_multipose_face2/weights/best.pt")

model.export(
    format="tflite",        
    optimize=True,          
    imgsz=320,              
    dynamic=False           
)

print("✅ Model berhasil diekspor ke TFLite dengan quantization FP16!")
